## Burned area delineation

In [1]:
workflow = dict([('id', 'burned-area-delineation'),
                ('label', 'Burned area delineation'),
                ('doc', 'Burned area delineation using two techniques')])


In [2]:
pre_event = dict([('id', 'pre_event'), 
                  ('label', 'Pre-event product for burned area delineation'),
                  ('doc', 'Pre-event product for burned area delineation'),
                  ('value', '/workspace/ogc-tb16/wdir/vi-pre/srutfvn8'), 
                  ('type', 'Directory')])

In [3]:
post_event = dict([('id', 'post_event'), 
                  ('label', 'Post-event product for burned area delineation'),
                  ('doc', 'Post-event product for burned area delineation'),
                  ('value', '/workspace/ogc-tb16/wdir/vi-post/rz7p3e6k'), 
                  ('type', 'Directory')])

In [4]:
ndvi_threshold = dict([('id', 'ndvi_threshold'),
                       ('value', '0.19'),
                       ('label', 'NDVI difference threshold'),
                       ('doc', 'NDVI difference threshold'),
                       ('type', 'string')]) 

In [5]:
ndwi_threshold = dict([('id', 'ndwi_threshold'),
                       ('value', '0.18'),
                       ('label', 'NDWI difference threshold'),
                       ('doc', 'NDWI difference threshold'),
                       ('type', 'string')]) 

In [6]:
import os
import sys
import gdal
import numpy as np
import logging
from pystac import *
from time import sleep
from helpers import *
import shutil

gdal.UseExceptions()

if not 'PREFIX' in os.environ.keys():
    
    os.environ['PREFIX'] = '/opt/anaconda/envs/env_vi/'

os.environ['GDAL_DATA'] =  os.path.join(os.environ['PREFIX'], 'share/gdal')
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
pre_event_cat = Catalog.from_file(os.path.join(pre_event['value'], 'catalog.json'))
post_event_cat = Catalog.from_file(os.path.join(post_event['value'], 'catalog.json'))

In [9]:
pre_event_item = next(pre_event_cat.get_items())

In [10]:
asset_pre_ndvi, asset_pre_ndwi, asset_pre_nbr = get_assets(pre_event_item)
asset_post_ndvi, asset_post_ndwi, asset_post_nbr = get_assets(pre_event_item)

In [11]:
asset_pre_ndvi, asset_pre_ndwi, asset_pre_nbr

('/workspace/ogc-tb16/wdir/vi-pre/srutfvn8/INDEX_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/NDVI_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.tif',
 '/workspace/ogc-tb16/wdir/vi-pre/srutfvn8/INDEX_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/NDWI_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.tif',
 '/workspace/ogc-tb16/wdir/vi-pre/srutfvn8/INDEX_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/NBR_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.tif')

In [12]:
asset_post_ndvi, asset_post_ndwi, asset_post_nbr

('/workspace/ogc-tb16/wdir/vi-pre/srutfvn8/INDEX_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/NDVI_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.tif',
 '/workspace/ogc-tb16/wdir/vi-pre/srutfvn8/INDEX_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/NDWI_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.tif',
 '/workspace/ogc-tb16/wdir/vi-pre/srutfvn8/INDEX_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/NBR_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.tif')

In [13]:
scaling_factor = 1/10000

scaling_factor

0.0001

If NDWI i2 - NDWI i1 > 0.18 and If NDVI i2 - NDVI i1 > 0.19 then burned pixels

In [14]:
_mem = '/vsimem/mem.tif'

Process NDVI difference

In [15]:
temp_ds = gdal.Translate(_mem,
                         asset_pre_ndvi,
                         outputType=gdal.GDT_Int16)

width = temp_ds.RasterXSize
height = temp_ds.RasterYSize
geo_transform = temp_ds.GetGeoTransform()
geo_ref = temp_ds.GetProjectionRef()

pre_ndvi = temp_ds.ReadAsArray()

temp_ds = None

In [16]:
temp_ds = gdal.Translate(_mem,
                         asset_post_ndvi,
                         outputType=gdal.GDT_Int16)

post_ndvi = temp_ds.ReadAsArray()

temp_ds = None

In [17]:
delta_ndvi = ((pre_ndvi - post_ndvi) * scaling_factor).astype(float)

pre_ndvi = post_ndvi = None

Process NDWI difference

In [18]:
temp_ds = gdal.Translate(_mem,
                         asset_pre_ndwi,
                         outputType=gdal.GDT_Int16)

pre_ndwi = temp_ds.ReadAsArray()

temp_ds = None

In [19]:
temp_ds = gdal.Translate(_mem,
                         asset_pre_ndwi,
                         outputType=gdal.GDT_Int16)

post_ndwi = temp_ds.ReadAsArray()

temp_ds = None

In [20]:
delta_ndwi = ((pre_ndwi - post_ndwi) * scaling_factor).astype(float)

pre_ndwi = post_ndwi = None

Burned area delineation

In [21]:
burned = np.where(((delta_ndwi  > float(ndwi_threshold['value'])) & (delta_ndvi > float(ndvi_threshold['value']))), 1, 0) 

In [22]:
output_name = 'DELINEATION'

In [23]:
default_bands = [{'name': 'dnbr',
                  'common_name': 'dNBR'}, 
                 {'name': 'tvi',
                  'common_name': 'tvi'}]

In [24]:
temp_name = '_BURNED_NDVI_NDWI_THRESHOLD.tif'
output_name = 'BURNED_NDVI_NDWI_THRESHOLD.tif'

driver = gdal.GetDriverByName('GTiff')

output = driver.Create(temp_name, 
                       width, 
                       height, 
                       1, 
                       gdal.GDT_Byte)

output.SetGeoTransform(geo_transform)
output.SetProjection(geo_ref)
output.GetRasterBand(1).WriteArray(burned),

output.FlushCache()

sleep(5)

output = None

del(output)

cog(temp_name, output_name)

In [25]:
results = {}

results['tvi'] = output_name


relativized burn ratio (RBR) 

In [26]:
temp_ds = gdal.Translate(_mem,
                         asset_pre_nbr,
                         outputType=gdal.GDT_Int16)

pre_nbr = temp_ds.ReadAsArray()

temp_ds = None

In [27]:
temp_ds = gdal.Translate(_mem,
                         asset_post_nbr,
                         outputType=gdal.GDT_Int16)

post_nbr = temp_ds.ReadAsArray()

temp_ds = None

In [28]:
delta_nbr = ((pre_nbr  - post_nbr) * scaling_factor).astype(float)

post_nbr = None

In [29]:
rbr = delta_nbr / (pre_nbr * scaling_factor + 1.001)

delta_nbr = pre_nbr = None

In [30]:
temp_name = '_BURNED_RBR.tif'
output_name = 'BURNED_RBR.tif'

driver = gdal.GetDriverByName('GTiff')

output = driver.Create(temp_name, 
                       width, 
                       height, 
                       1, 
                       gdal.GDT_Float32)

output.SetGeoTransform(geo_transform)
output.SetProjection(geo_ref)
output.GetRasterBand(1).WriteArray(rbr),

output.FlushCache()

sleep(5)

output = None

del(output)

cog(temp_name, output_name)

In [31]:
results['dnbr'] = output_name

In [32]:
catalog = Catalog(id='catalog',
                      description='Results')

catalog.clear_items()
catalog.clear_children()

<Catalog id=catalog>

In [34]:
item_name = 'BURNED_AREA_DELINEATION'


pre_event_item.properties.pop('eo:bands', None)

result_item = Item(id=item_name,
                   geometry=pre_event_item.geometry,
                   bbox=pre_event_item.bbox,
                   datetime=pre_event_item.datetime,
                   properties=pre_event_item.properties)

result_item.common_metadata.set_gsd(10)

eo_item = extensions.eo.EOItemExt(result_item)

In [35]:
results

{'tvi': 'BURNED_NDVI_NDWI_THRESHOLD.tif', 'dnbr': 'BURNED_RBR.tif'}

In [36]:
default_bands = {'dnbr': {'name': 'dnbr', 'common_name': 'dNBR'}, 
                 'tvi':  {'name': 'tvi', 'common_name': 'tvi'}}

In [37]:
bands = []
   
os.makedirs(os.path.join('.', item_name),
                exist_ok=True)

for key, value in results.items():

    print(key, value)
    
    

    result_item.add_asset(key=key,
                          asset=Asset(href='./{}'.format(value), 
                                    media_type=MediaType.GEOTIFF))

    asset = result_item.get_assets()[key]                                   

    stac_band = extensions.eo.Band.create(name=key, 
                                           common_name=default_bands[key]['common_name'],
                                                description=default_bands[key]['name'])
    bands.append(stac_band)

    eo_item.set_bands([stac_band], asset=asset)

    shutil.move(value, os.path.join('.', item_name, value))
    
eo_item.set_bands(bands)

eo_item.apply(bands)  

tvi BURNED_NDVI_NDWI_THRESHOLD.tif
dnbr BURNED_RBR.tif


In [38]:
catalog.add_items([result_item])
    
catalog.normalize_and_save(root_href='./',
                           catalog_type=CatalogType.SELF_CONTAINED)

In [39]:
catalog.describe()

* <Catalog id=catalog>
  * <Item id=BURNED_AREA_DELINEATION>
